## Post-block Assignment 1

In [1]:
!pip install nbformat

  Obtaining dependency information for nbformat from https://files.pythonhosted.org/packages/a9/82/0340caa499416c78e5d8f5f05947ae4bc3cba53c9f038ab6e9ed964e22f1/nbformat-5.10.4-py3-none-any.whl.metadata
  Obtaining dependency information for fastjsonschema>=2.15 from https://files.pythonhosted.org/packages/90/2b/0817a2b257fe88725c25589d89aec060581aabf668707a8d03b2e9e0cb2a/fastjsonschema-2.21.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/78.5 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/78.5 kB ? eta -:--:--
   --------------- ------------------------ 30.7/78.5 kB 262.6 kB/s eta 0:00:01
   -------------------- ------------------- 41.0/78.5 kB 196.9 kB/s eta 0:00:01
   ------------------------------- -------- 61.4/78.5 kB 252.2 kB/s eta 0:00:01
   ---------------------------------------- 78.5/78.5 kB


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: C:\Users\EMMANUEL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!pip install -Uqq plotly

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
PLOTLY_THEME = "plotly_dark"

## 1. Read the data

In [4]:
def read_data(df):

    df = pd.read_csv(df)
    df.columns = df.columns.str.lower().str.replace(' ', '_') # Ensure all columns follow rules of declaring variables in coding
    df = df.drop(axis=1, columns=['unnamed:_0', 'index'])
    return df

df = read_data(df = "data/DataScientist (1).csv")
print(df.shape)
df.sample(5)

(3909, 15)


,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,type_of_ownership,industry,sector,revenue,competitors,easy_apply
1569,Medical Laboratory Scientist (MLS),$84K-$101K (Glassdoor est.),Job Description\r\nMedical Laboratory Scientis...,4.1,APN Staffing & Employment Solutions\r\n4.1,"Gilbert, AZ","Phoenix, AZ",1 to 50 employees,1991,Company - Private,Staffing & Outsourcing,Business Services,$500 million to $1 billion (USD),-1,-1
1789,ML-Data Science Engineer with OCR,$46K-$83K (Glassdoor est.),ML Data Scientists ML Data Scientist shall pro...,4.3,Peoplentech\r\n4.3,"Philadelphia, PA","Vienna, VA",1 to 50 employees,-1,Contract,IT Services,Information Technology,$1 to $5 million (USD),-1,-1
3146,"AI/ML - Sr Data Scientist, Siri Product Strategy",$121K-$192K (Glassdoor est.),"Posted: Jun 26, 2020\r\nWeekly Hours: 40\r\nRo...",4.1,Apple\r\n4.1,"Santa Clara, CA","Cupertino, CA",10000+ employees,1976,Company - Public,Computer Hardware & Software,Information Technology,$10+ billion (USD),"Google, Microsoft, Samsung Electronics",-1
1691,Quantum Principal Data Scientist (Industrial-P...,$44K-$80K (Glassdoor est.),"Introduction\r\nAs a Data Scientist at IBM, yo...",3.7,IBM\r\n3.7,"Philadelphia, PA","Armonk, NY",10000+ employees,1911,Company - Public,IT Services,Information Technology,$10+ billion (USD),"Amazon, Accenture, Microsoft",-1
2103,COMPUTER SCIENTIST - ENGINEER - RESEARCH ENGIN...,$43K-$76K (Glassdoor est.),Join a team that develops creative solutions f...,4.1,Southwest Research Institute\r\n4.1,"San Antonio, TX","San Antonio, TX",1001 to 5000 employees,1947,Nonprofit Organization,Research & Development,Business Services,$500 million to $1 billion (USD),"Los Alamos National Laboratory, Battelle, SRI ...",-1


--- 

## 2. Data Preprocessing
- This entails data cleaning, extracting features, imputing missing values, remove noise etc

In [5]:
def clean_salary_estimates(df, column_name='salary_estimate'):
    
    df[column_name] = df[column_name].str.replace(r'\(.*\)', '', regex=True)  # Clean the salary_estimate column by removing extra text in parentheses,remove extra text

    # Remove '$' and 'K', then split by '-'
    df[['min_salary_estimate', 'max_salary_estimate']] = df[column_name].str.replace(r'[^\d-]', '', regex=True) \
                                                     .str.split('-', expand=True)
    # Replace 'K' with '000' to convert to actual values
    df['min_salary_estimate'] = df['min_salary_estimate'].str.replace('K', '').astype(int) * 1000
    df['max_salary_estimate'] = df['max_salary_estimate'].str.replace('K', '').astype(int) * 1000

    return df

In [15]:
def split_revenue(revenue):
    if "to" in revenue:
        return revenue.replace(" (USD)", "").split(" to ")
    elif "Less than" in revenue:
        return ["0", revenue.replace("Less than ", "").replace(" (USD)", "")]
    elif "Unknown" in revenue or revenue == "-1":
        return [None, None]
    elif "$10+ billion" in revenue:
        return ["$10 billion", None]  # Assuming $10+ billion is minimum at 10B
    else:
        return [None, None]

# Apply function to create min_revenue and max_revenue columns
df[["min_revenue", "max_revenue"]] = pd.DataFrame(df["revenue"].apply(split_revenue).tolist(), index=df.index)

In [16]:
def classify_size(size):
    """Classify company size based on employee count."""
    if isinstance(size, str):  # Ensure it's a string
        size = size.replace(" employees", "").strip()
        
        if size == "-1":
            return "Unknown"
        elif "10000+" in size:
            return "Large"
        elif " to " in size:
            min_size, max_size = map(int, size.split(" to "))
            if max_size <= 200:
                return "Small"
            elif max_size <= 1000:
                return "Medium"
            else:
                return "Large"
            
        elif size.isdigit():  # Single number case
            num = int(size)
            if num <= 200:
                return "Small"
            elif num <= 1000:
                return "Medium"
            else:
                return "Large"
    
    return "Unknown"

# Apply company size classification
df["company_size_category"] = df["size"].apply(classify_size)

In [17]:
df = df[~df['sector'].str.contains(r'\d', na=False)] # Remove numbers on feature sector
df = clean_salary_estimates(df)
df.sample(4)

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,type_of_ownership,industry,sector,revenue,competitors,easy_apply,min_revenue,max_revenue,company_size_category,min_salary_estimate,max_salary_estimate
1409,"Analytics Manager, Insurance Operations",$45K-$81K,"Hi, we're Oscar. We're hiring an Analytics Man...",3.7,Oscar Health\r\n3.7,"Tempe, AZ","New York, NY",1001 to 5000 employees,2012,Company - Private,Insurance Agencies & Brokerages,Insurance,$2 to $5 billion (USD),-1,True,$2,$5 billion,Large,45000,81000
1916,SR BUSINESS DATA ANALYST PRN,$55K-$101K,Short Description\r\n\r\nWork with business le...,3.2,Cooper University Health Care\r\n3.2,"Camden, NJ","Camden, NJ",5001 to 10000 employees,1887,Hospital,Health Care Services & Hospitals,Health Care,Unknown / Non-Applicable,-1,-1,None,None,Large,55000,101000
1866,SQL Data Analyst,$143K-$237K,Piper Companies is looking for an SQL Data Ana...,3.9,"Zachary Piper Solutions, LLC\r\n3.9","Horsham, PA","Mc Lean, VA",51 to 200 employees,2011,Company - Private,Enterprise Software & Network Solutions,Information Technology,$10 to $25 million (USD),-1,-1,$10,$25 million,Small,143000,237000
2643,Quantum Principal Data Scientist- Industrial P...,$85K-$159K,"IntroductionAs a Data Scientist at IBM, you wi...",3.7,IBM Corporation\r\n3.7,"Dallas, TX","Armonk, NY",10000+ employees,1911,Company - Public,IT Services,Information Technology,$10+ billion (USD),"Amazon, Accenture, Microsoft",-1,$10 billion,None,Large,85000,159000


---

<div style="text-align:center; font-size:30px; font-weight:bold;"> Derive insights of strategic value</div>

## Salary By Sector

In [ ]:
# fig = px.box(data_frame=df, x = "sector", y = "min_salary_estimate")
# fig.show()

### Number of job posting by location

In [18]:
roles_by_sectors = df.groupby(['sector'])[['job_title']]\
                     .count()\
                     .reset_index()\
                     .sort_values(by='job_title', ascending=False)\
                     .head(10)
                     
roles_by_sectors = roles_by_sectors.rename(columns={'job_title': 'num_of_job_posting'})

fig = px.bar(roles_by_sectors, x = "sector", y = "num_of_job_posting", template=PLOTLY_THEME, title="Number of Job Posting by Sector")
fig.update_layout(title_x = 0.5)
fig.show()

In [ ]:
df.head()
df[df.sector  == "Business Services"][["sector", "industry"]]
 


1. High Demand for Data Science in IT and Business Services

- The **`Information Technology (IT)`** sector has the highest number of job postings. This suggests a strong demand for data science professionals in software development, AI, and cloud computing.

- **`Business Services`** is a Major Employer: The second-largest sector is Business Services, including consulting,advertising and marketing, staffing firms, which highlights outsourcing trends.

- Growing Demand in Biotech & Pharmaceuticals: With AI-driven healthcare analytics and drug discovery, data science hiring in Biotech & Pharma is becoming more prominent.

- Finance & Healthcare Are Expanding: Traditional industries such as Finance and Health Care have significant hiring activity, signaling an increasing reliance on data-driven decision-making in risk assessment, fraud detection, and patient care optimization.

- Emerging Sectors (Insurance, Education, Government, Media, Manufacturing): While these sectors have fewer postings compared to IT and Business Services, they represent growth areas where data-driven solutions are being integrated.

2. Strategic Recommendations for Investors & Recruitment Firms

#### For Investors:
- Investment in Data Science Talent Platforms: Given the high demand across industries, investing in AI-driven recruitment platforms tailored for data science roles can be lucrative.

- Targeted Training & Upskilling Platforms: The rise of data science in non-tech sectors (e.g., Healthcare, Finance, Manufacturing) presents an opportunity to invest in industry-specific data science training platforms.

#### For Recruitment Companies:
- `Niche Recruiting for High-Demand Sectors`: Since IT, Business Services, and Finance are major hirers, recruiters should build specialized pipelines for these industries.

#### Data Monetization Strategies

Strategy	Monetization Approach
Recruitment Data Dashboards	Offer subscription-based dashboards for companies to track real-time hiring trends in data science
Industry-Specific Job Boards	Build custom job boards focused on Finance, Biotech, and Healthcare, sectors where specialized hiring is growing
AI-Driven Candidate Matching	Develop a machine learning-based recruitment tool to match job seekers with the most suitable industry roles
Corporate Workforce Planning Insights	Sell industry-specific hiring forecasts to businesses so they can optimize workforce planning
Data Science Salary Benchmarking	Provide premium salary analytics for companies to benchmark compensation trends by industry
Hiring Strategy Consulting	Offer consulting packages to large companies and startups that need data science recruitment expertise









# References:
- https://www.cobloom.com/careers-blog/data-science-salaries-how-much-can-you-really-earn#:~:text=Some%20sectors%E2%80%94like%20healthcare%2C%20autonomous%20vehicles%2C,both%20pay%20and%20professional%20fulfillment



---

In [22]:
roles_by_company_size = df.groupby(['company_size_category'])[['job_title']]\
                          .count()\
                          .reset_index()\
                          
roles_by_company_size = roles_by_company_size.rename(columns={"job_title": "num_of_job_posting"})                          
                          
roles_by_company_size.head()

,company_size_category,num_of_job_posting
0,Large,1738
1,Medium,696
2,Small,914
3,Unknown,15


In [20]:
fig = px.pie(roles_by_company_size, values='num_of_job_posting', names="company_size_category", template=PLOTLY_THEME)
fig.show()

In [21]:
df

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,type_of_ownership,industry,sector,revenue,competitors,easy_apply,min_revenue,max_revenue,company_size_category,min_salary_estimate,max_salary_estimate
0,Senior Data Scientist,$111K-$181K,"ABOUT HOPPER\r\n\r\nAt Hopper, we’re on a miss...",3.5,Hopper\r\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1,None,None,Medium,111000,181000
1,"Data Scientist, Product Analytics",$111K-$181K,"At Noom, we use scientifically proven methods ...",4.5,Noom US\r\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1,None,None,Large,111000,181000
3,Data Analyst,$111K-$181K,Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\r\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades",-1,None,None,Medium,111000,181000
4,"Director, Data Science",$111K-$181K,"Director, Data Science - (200537)\r\nDescripti...",3.4,United Entertainment Group\r\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5",-1,None,None,Small,111000,181000
5,Data Scientist,$111K-$181K,Job Brief\r\n\r\nThe ideal candidate will have...,2.9,IFG Companies\r\n2.9,"New York, NY","Hartford, CT",201 to 500 employees,1985,Company - Private,Insurance Carriers,Insurance,Unknown / Non-Applicable,"Colony Specialty, Markel, RLI",-1,None,None,Medium,111000,181000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3904,AWS Data Engineer,$55K-$112K,About Us\r\n\r\nTachyon Technologies is a Digi...,4.4,Tachyon Technologies\r\n4.4,"Dublin, OH","Irving, TX",201 to 500 employees,2011,Company - Private,IT Services,Information Technology,$10 to $25 million (USD),-1,-1,$10,$25 million,Medium,55000,112000
3905,Data Analyst â Junior,$55K-$112K,"Job description\r\nInterpret data, analyze res...",5.0,"Staffigo Technical Services, LLC\r\n5.0","Columbus, OH","Woodridge, IL",51 to 200 employees,2008,Company - Private,IT Services,Information Technology,$50 to $100 million (USD),-1,-1,$50,$100 million,Small,55000,112000
3906,Security Analytics Data Engineer,$55K-$112K,Job DescriptionThe Security Analytics Data Eng...,3.8,"PDS Tech, Inc.\r\n3.8","Dublin, OH","Irving, TX",5001 to 10000 employees,1977,Company - Private,Staffing & Outsourcing,Business Services,$100 to $500 million (USD),-1,-1,$100,$500 million,Large,55000,112000
3907,Security Analytics Data Engineer,$55K-$112K,The Security Analytics Data Engineer will inte...,4.0,Data Resource Technologies\r\n4.0,"Dublin, OH","Omaha, NE",1 to 50 employees,-1,Company - Private,Accounting,Accounting & Legal,Less than $1 million (USD),-1,-1,0,$1 million,Small,55000,112000


##### Provide insights

#### For Investors:
- Invest in AI-powered Recruitment Solutions for Large Enterprises
   Given that large corporations dominate hiring (51.7%), investing in data-driven recruitment platforms (e.g., AI resume screening, automated job matching, and workforce analytics) can be highly lucrative.

- Support Recruitment Marketplaces for Startups & SMBs: Startups and small businesses (27.2%) need fast and cost-effective hiring solutions.
Investing in niche job boards or AI-driven applicant tracking systems (ATS) focused on startups can generate subscription revenue.

---

#### Get the avg min salary estimate by sector and location

In [ ]:

avg_min_salary_by_sector = df.groupby(['sector','location'])[['min_salary_estimate']]\
                             .mean()\
                             .reset_index()\
                             .sort_values(by=['min_salary_estimate'], ascending=False)\
                             .round(2)
	

#print(avg_min_salary_by_sector.head(10)) # Top 10 of most paying sectors by sector and location

fig = px.bar(data_frame=avg_min_salary_by_sector.head(10).sort_values(by='sector', ascending=False),
              x = "sector",
              y = "min_salary_estimate",
              color="location",
              title="Average min salary by sector and location", 
              template=PLOTLY_THEME)
fig.update_layout(title_x=0.5)
fig.show()

## 1. High-Paying Sectors and Locations
- Finance stands out as the sector with the highest minimum salary estimates, particularly in Santa Clara, CA.
- Information Technology (IT), a major sector for data science roles, shows a strong salary offering, particularly in Redwood City, CA.
- Real Estate, Health Care, and Business Services also offer competitive salaries, indicating potential demand for data analytics and AI-driven decision-making.


## 2. Data Science Job Monetization Opportunity
- The Finance and IT sectors are particularly lucrative for data science professionals, making them prime targets for recruitment solutions.
- Santa Clara and Redwood City, CA, seem to be key locations offering high-paying jobs, meaning job postings from these locations could be prioritized for insights and monetization.

---

#### Demand for Data Science Roles by Company Size

In [23]:
demand_by_company_size = df.groupby(['industry', 'company_size_category'])[['job_title']]\
                      .count()\
                      .reset_index()\
                      .sort_values(by = "job_title", ascending=False)

demand_by_company_size.head(10)

,industry,company_size_category,job_title
20,Biotech & Pharmaceuticals,Large,173
105,IT Services,Small,171
103,IT Services,Large,160
95,Health Care Services & Hospitals,Large,151
115,Internet,Large,148
104,IT Services,Medium,140
37,Computer Hardware & Software,Large,131
179,Staffing & Outsourcing,Small,121
177,Staffing & Outsourcing,Large,104
39,Computer Hardware & Software,Small,102


In [24]:
#Salary Trends Based on Company Size

avg_min_salary_by_comany_size = df.groupby(['company_size_category'])[['max_salary_estimate']]\
                                  .median()\
                                  .reset_index()\
                                  .round(2)


avg_min_salary_by_comany_size

,company_size_category,max_salary_estimate
0,Large,136000.0
1,Medium,122000.0
2,Small,139000.0
3,Unknown,149000.0


In [25]:
fig = px.bar(data_frame=avg_min_salary_by_comany_size,
             x='company_size_category', 
             y='max_salary_estimate')
fig.show()

### How many Job Posting by Industry and Location?

In [27]:
roles_by_industry = df.groupby(['industry', 'company_size_category'])[['job_title']]\
                      .count()\
                      .reset_index()\
                      .sort_values(by = "job_title", ascending=False)

In [26]:
fig = px.bar(data_frame=roles_by_industry.head(10), x = "industry",
                                                    y = "job_title",
                                                    color = "company_size_category",
                                                    title="Number of Job Posting by location and industry", 
                                                    template=PLOTLY_THEME)
fig.update_layout(title_x=0.5)
fig.show()

NameError: name 'roles_by_industry' is not defined